In [ ]:
import pyspark
from pyspark.sal import SparkSession
from pyspark.sal.functions import split
from pyspark.sal.functions import explode

In [ ]:
spark = SparkSession \
    .builder.appName('StructuedNetworkWorkCount') \
    .getOrCreate()
spark.sparkContext.setLogLevel('WARN')

In [ ]:
lines = spark.readStream \
    .format('socket') \
    .option('host','localhost') \
    .option('port',9999) \
    .load()

In [ ]:
words = lines.select(
    explode(
        split(lines.value,' ')
    ).alias('word')
)
wordCounnt = words.groupBy('word').count()

In [ ]:
query = wordCounnt \
    .writeStream \
    .outputMode('complete') \
    .format('console') \
    .trigger(processingTime='8 seconds') \
    .start()
query.awaitTermination()